In [1]:
import os
path = os.path.join('/home/santiago/Documents/dev/reservoirpy')
import sys
sys.path.insert(0,path)
import pandas as pd
import numpy as np
from reservoirpy.materialbalancepy import mbe as mb
from reservoirpy.krpy import kr_df
from reservoirpy.pvtpy import black_oil as bl
import matplotlib.pyplot as plt

In [2]:
data = np.array([
    [0.1000000, 0.0000000, 1.0000000, 0],
    [0.2000000, 0.0138408, 0.6869530, 0],
    [0.3000000, 0.0553633, 0.4471809, 0],
    [0.4000000, 0.1245675, 0.2709139, 0],
    [0.5000000, 0.2214533, 0.1483818, 0],
    [0.6000000, 0.3460208, 0.0698148, 0],
    [0.7000000, 0.4982699, 0.0254427, 0],
    [0.8000000, 0.6782007, 0.0054956, 0],
    [0.9000000, 0.8858131, 0.0002035, 0],
    [0.9500000, 1.0000000, 0.0000000, 0],
    [1.0000000, 1.0000000, 0.0000000, 0],
])
krwo = kr_df(data,columns=['sw','krw','kro','pc'])
krwo

,krw,kro,pc
sw,,,
0.10,0.000000,1.000000,0.0
0.20,0.013841,0.686953,0.0
0.30,0.055363,0.447181,0.0
0.40,0.124567,0.270914,0.0
0.50,0.221453,0.148382,0.0
0.60,0.346021,0.069815,0.0
0.70,0.498270,0.025443,0.0
0.80,0.678201,0.005496,0.0
0.90,0.885813,0.000204,0.0


In [3]:
data = np.array([
    [0.81, 0.018],
    [0.76, 0.063],
    [0.60, 0.850],
    [0.50, 3.350],
    [0.40, 10.2]

])
krgo_ratio = kr_df(data,columns=['so','krg_kro'],index='so')
krgo_ratio

,krg_kro
so,
0.81,0.018
0.76,0.063
0.60,0.850
0.50,3.350
0.40,10.200


In [4]:
krgo_ratio.interpolate(0.50001)

,krg_kro
saturation,
0.50001,3.34975


In [5]:
# Oil PVT
oil_pvt = bl.pvt({
    'pressure': [2925,2100,1800,1500],
    'bo':[1.429,1.480,1.468,1.440],
    'bt':[1.429,1.480,1.559,1.792],
    'rs':[1340,1340,1280,1150],
    'bg':[0.001,0.001283,0.001518,0.00183],
    'muo':[10,7,7.5,8],
    'mur':[0,34.1,38.3,42.4]
})

formation = 'fm1'
api = 25
pb = 2100
rsi = 1340
temp = 175

o = bl.oil(
    formation=formation,
    api=api,
    pb=pb,
    rsb=rsi,
    temp=temp,
    pvt=oil_pvt
)
o.pvt

,bo,bt,rs,bg,muo,mur
pressure,,,,,,
2925,1.429,1.429,1340,0.001000,10.0,0.0
2100,1.480,1.480,1340,0.001283,7.0,34.1
1800,1.468,1.559,1280,0.001518,7.5,38.3
1500,1.440,1.792,1150,0.001830,8.0,42.4


In [6]:
# GasPVT
gas_pvt = bl.pvt({
    'pressure': [2925,2100,1800,1500],
    'bg':[0.001,0.001283,0.001518,0.00183],
    'mug':[0,0.205,0.1958,0.1886]
})

formation = 'fm1'

temp = 175
g = bl.gas(
    formation=formation,
    pvt = gas_pvt,
    temp=temp
)
g.pvt

,bg,mug
pressure,,
2925,0.001000,0.0000
2100,0.001283,0.2050
1800,0.001518,0.1958
1500,0.001830,0.1886


In [7]:
# Water

w = bl.water(
    formation=formation,
    pb = pb,
    temp = temp)

w.pvt_from_correlations(start_pressure=1000,end_pressure=3000,n=5)
w.pvt['cw']=3.6e-6
w.pvt

,rsw,cw,bw,muw,rhow
pressure,,,,,
1000.0,6.803417,0.000004,1.029221,0.375964,60.628376
1500.0,8.975257,0.000004,1.028531,0.375964,60.669053
2000.0,11.044822,0.000004,1.027709,0.375964,60.717562
2500.0,13.012112,0.000004,1.026144,0.375964,60.810184
3000.0,14.877126,0.000004,1.024409,0.375964,60.913157


In [ ]:
swi=0.15
cf=4.9e-6
ooip=10e6

In [ ]:
res1 = mb.oil_reservoir(
    n=ooip,
    swi=swi,
    cf = cf,
    oil = o,
    water = w,
    gas = g,
    pi=2925,    
    kr_wo = krwo,
    kr_go = krgo_ratio
)


In [ ]:
r = np.linspace(2100,2900,10)[::-1]

r

In [ ]:
res1.oil.pvt.interpolate([2925,2900])

In [ ]:
f = res1.forecast_np(r,wp=False)
print(f.tail())

In [ ]:
new_r = np.linspace(1500,2800,5)[::-1]
new_r = [2100,1800,1500]
f_new = res1.forecast_np(new_r,wp=False)
print(f_new)

fig, ax = plt.subplots(2,1, figsize=(12,6))
ax[0].scatter(f_new['np'],f_new.index, color='green')
ax[1].scatter(f_new['gp']/1e9,f_new.index, color='red')

In [ ]:
2.16e9/1e6

In [ ]:
f=res1.forecast_np(r,wp=True, er_np=0.01)
f

In [ ]:
f_wp = res1.forecast_np(r,wp=True)
print(f_wp.tail())

In [ ]:
fig, ax = plt.subplots()
ax.plot(f['np'], f.index, label='without wp')
ax.plot(f_wp['np'], f_wp.index, label='with wp')
ax.set_ylim(0,3000)
ax.legend()

In [ ]:
aq = mb.pot_aquifer(k=9987)

In [ ]:
res1_aq = mb.oil_reservoir(
    n=ooip,
    swi=swi,
    cf = cf,
    oil = o,
    water = w,
    pi=2925,    
    kr_wo = kr,
    aquifer = aq
)

In [ ]:
f_aq= res1_aq.forecast_np(r,wp=True)
f_aq

In [ ]:
plt.plot(f_aq['np'],f_aq['wor'])
plt.yscale('log')

In [ ]:
kr.plot()

In [ ]:
plt.plot(f_aq['sw'],f_aq['bsw'])